In [7]:
from tensorflow.keras.applications import VGG16

# mobilenet
# efficientnetB1~B6

model_base = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(150,150,3))

model_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [6]:
# 개와 고양이 training data set에 대한 feature map을 추출해요!

import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = '/content/drive/MyDrive/Machine Learning Colab/CAT_DOG/cat_dog_small'
train_dir = os.path.join(base_dir,'train')
validation_dir = os.path.join(base_dir,'validation')

datagen = ImageDataGenerator(rescale=1/255)
batch_size=20

def extract_feature(directory, sample_count):
    features = np.zeros(shape=(sample_count,4,4,512))
    labels= np.zeros(shape=(sample_count,))

    generator = datagen.flow_from_directory(
        directory,
        target_size=(150,150),
        batch_size=batch_size,
        class_mode='binary'
    )

    i = 0

    for x_data_batch, t_data_batch in generator:
        feature_batch = model_base.predict(x_data_batch)
        features[i*batch_size:(i+1)*batch_size] = feature_batch
        labels[i*batch_size:(i+1)*batch_size] = t_data_batch

        i += 1
        if i*batch_size >= sample_count:
            break

    return features, labels

train_features, train_labels = extract_feature(train_dir, 2000)    
validation_features, validation_labels = extract_feature(validation_dir, 1000)    


Found 2004 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [9]:
train_features = np.reshape(train_features, (2000,4*4*512))
validation_features = np.reshape(validation_features, (1000,4*4*512))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(256,
                activation='relu',
                input_shape=(4*4*512,)))
model.add(Dropout(0.5))
model.add(Dense(1,
                activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_features,
                    train_labels,
                    epochs=30,
                    batch_size=64,
                    validation_data=(validation_features,validation_labels))

Epoch 1/30
32/32 [==============================] - 1s 10ms/step - loss: 0.6806 - accuracy: 0.6116 - val_loss: 0.3746 - val_accuracy: 0.8420
Epoch 2/30
32/32 [==============================] - 0s 5ms/step - loss: 0.3639 - accuracy: 0.8317 - val_loss: 0.3089 - val_accuracy: 0.8720
Epoch 3/30
32/32 [==============================] - 0s 5ms/step - loss: 0.2940 - accuracy: 0.8781 - val_loss: 0.2801 - val_accuracy: 0.8890
Epoch 4/30
32/32 [==============================] - 0s 5ms/step - loss: 0.2231 - accuracy: 0.9123 - val_loss: 0.2663 - val_accuracy: 0.8930
Epoch 5/30
32/32 [==============================] - 0s 5ms/step - loss: 0.2233 - accuracy: 0.9149 - val_loss: 0.2496 - val_accuracy: 0.9000
Epoch 6/30
32/32 [==============================] - 0s 5ms/step - loss: 0.1870 - accuracy: 0.9360 - val_loss: 0.2403 - val_accuracy: 0.8980
Epoch 7/30
32/32 [==============================] - 0s 5ms/step - loss: 0.1683 - accuracy: 0.9397 - val_loss: 0.2373 - val_accuracy: 0.9110
Epoch 8/30
32/32 [=